In [1]:
import pandas as pd
from simpledbf import Dbf5
import numpy as np
from os import listdir
from os.path import isfile, join
from functions.cargar_data import *

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
# carga personas
personas96 = pd.read_csv('marcos_censales/peronas_zona_96.csv')
personas96.head(2)

,codcomp,personas
0,1001003001,10
1,1001003002,19


In [3]:
path = 'datos_ine_96/dbfs/'

In [4]:
# lee datos de Mvdeo.
dbf = Dbf5(path + '01 idgeo 96-04 mvd.dbf')
mvo = dbf.to_dataframe()
print(mvo.shape)
mvo.head(2)

(12165, 4)


,CCOMP96,CCOMP04,REG96A04,MODIF96A04
0,101001001,101001001,0,SIN MODIFICACION
1,101001003,101001003,0,SIN MODIFICACION


In [5]:
mvo.groupby('MODIF96A04').size()

MODIF96A04
FUSION               240
PARTICION           3153
SIN MODIFICACION    8772
dtype: int64

In [6]:
# arregla typo en data de Flores
dbf = Dbf5(path + '07 idgeo 96-04 flores.dbf')
flores = dbf.to_dataframe().rename({'CCOPM04': 'CCOMP04'}, axis=1)
print(flores.shape)
flores.head(2)

(724, 6)


,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04
0,701001001,701001001,0,SIN MODIFICACION,7320,7320
1,701001002,701001002,0,SIN MODIFICACION,7320,7320


In [8]:
# lee, enlista y convierte a DF
files = [f for f in listdir(path) if isfile(join(path, f))]
files.remove('01 idgeo 96-04 mvd.dbf')
files.remove('07 idgeo 96-04 flores.dbf')
df_list = [(Dbf5(path + i).to_dataframe()) for i in files]
df_list.append(flores)
interior = pd.concat(df_list)
print('Con Flores: ', interior.shape[0])
interior.head(5)

Con Flores:  48571


,CCOMP96,CCOMP04,REG96A04,MODIF96A04,CODLOC_96,CODLOC_04
0,1001013001,1001013001,0,SIN MODIFICACION,10320,10320.0
1,1001013002,1001013002,0,SIN MODIFICACION,10320,10320.0
2,1001013003,1001013003,0,SIN MODIFICACION,10320,10320.0
3,1001013004,1001013004,0,SIN MODIFICACION,10320,10320.0
4,1001013005,1001013005,0,SIN MODIFICACION,10320,10320.0


In [ ]:
# exporta
interior.to_csv('datos_ine_96/correspondencia_interior.csv', index=False, float_format='%.0f')
mvo.to_csv('datos_ine_96/correspondencia_mvdeo.csv', index=False)

In [18]:
dfs = pd.concat([interior.loc[:,['CCOMP96', 'CCOMP04', 'REG96A04', 'MODIF96A04']], mvo])

In [19]:
dfs.groupby('MODIF96A04').size()

MODIF96A04
FUSION               1199
Inf 2 de CPV96          5
PARTICION            9766
SIN MODIFICACION    49766
dtype: int64

In [20]:
len(dfs.CCOMP96.unique())

53292

In [21]:
dfs.head()

,CCOMP96,CCOMP04,REG96A04,MODIF96A04
0,1001013001,1001013001,0,SIN MODIFICACION
1,1001013002,1001013002,0,SIN MODIFICACION
2,1001013003,1001013003,0,SIN MODIFICACION
3,1001013004,1001013004,0,SIN MODIFICACION
4,1001013005,1001013005,0,SIN MODIFICACION


In [22]:
dfs_personas = dfs.merge(personas96, left_on='CCOMP96', right_on='codcomp')
dfs_personas.head(2)

,CCOMP96,CCOMP04,REG96A04,MODIF96A04,codcomp,personas
0,1001013001,1001013001,0,SIN MODIFICACION,1001013001,25
1,1001013002,1001013002,0,SIN MODIFICACION,1001013002,87


In [23]:
dfs_personas_sd = dfs_personas.drop_duplicates('CCOMP96')
dfs_personas_sd.head()

,CCOMP96,CCOMP04,REG96A04,MODIF96A04,codcomp,personas
0,1001013001,1001013001,0,SIN MODIFICACION,1001013001,25
1,1001013002,1001013002,0,SIN MODIFICACION,1001013002,87
2,1001013003,1001013003,0,SIN MODIFICACION,1001013003,119
3,1001013004,1001013004,0,SIN MODIFICACION,1001013004,86
4,1001013005,1001013005,0,SIN MODIFICACION,1001013005,107


In [24]:
parti = dfs.loc[dfs.MODIF96A04=='PARTICION']
parti.head(6)

,CCOMP96,CCOMP04,REG96A04,MODIF96A04
179,1001038900,1001038800,1,PARTICION
180,1001038001,1001038801,1,PARTICION
181,1001038900,1001038802,1,PARTICION
182,1001038001,1001038803,1,PARTICION
183,1001038018,1001038804,1,PARTICION
184,1001038018,1001038805,1,PARTICION


In [25]:
parti_group = parti.groupby('CCOMP96').size().to_frame(name='division')
parti_group = parti_group.merge(dfs_personas_sd, left_on='CCOMP96', right_on='codcomp')
parti_group['personas'] = round(parti_group['personas'] / parti_group['division']).astype(int)
parti_group.drop(['division', 'MODIF96A04'], axis=1, inplace=True)
parti_group.head()

,CCOMP96,CCOMP04,REG96A04,codcomp,personas
0,101003009,101003800,1,101003009,44
1,107205004,107205500,1,107205004,40
2,108002001,108002400,1,108002001,113
3,108002006,108002401,1,108002006,6
4,108008003,108008800,1,108008003,8


In [26]:
fusion = dfs_personas.loc[dfs_personas.MODIF96A04=='FUSION']
fusion_group = fusion.groupby('CCOMP04', as_index=False).sum('personas')
fusion_group.head(3)

,CCOMP04,CCOMP96,REG96A04,codcomp,personas
0,105102800,210204019,4,210204019,43
1,109007800,218014007,4,218014007,5
2,109008800,327024027,6,327024027,311


In [27]:
parti_fusion = pd.concat([parti_group, fusion_group])
parti_fusion.head()

,CCOMP96,CCOMP04,REG96A04,codcomp,personas
0,101003009,101003800,1,101003009,44
1,107205004,107205500,1,107205004,40
2,108002001,108002400,1,108002001,113
3,108002006,108002401,1,108002006,6
4,108008003,108008800,1,108008003,8


In [28]:
cols = parti_fusion.columns
sm = dfs_personas.loc[dfs_personas.MODIF96A04=='SIN MODIFICACION', cols]
sm.head()

,CCOMP96,CCOMP04,REG96A04,codcomp,personas
0,1001013001,1001013001,0,1001013001,25
1,1001013002,1001013002,0,1001013002,87
2,1001013003,1001013003,0,1001013003,119
3,1001013004,1001013004,0,1001013004,86
4,1001013005,1001013005,0,1001013005,107


In [29]:
sum(sm.personas)

2832917

In [30]:
todo = pd.concat([parti_fusion, sm])
todo.head()

,CCOMP96,CCOMP04,REG96A04,codcomp,personas
0,101003009,101003800,1,101003009,44
1,107205004,107205500,1,107205004,40
2,108002001,108002400,1,108002001,113
3,108002006,108002401,1,108002006,6
4,108008003,108008800,1,108008003,8


In [31]:
todo.shape

(46127, 5)

In [32]:
sum(todo.personas)

2961551

In [33]:
todo.to_csv('marcos_censales/personas_96_zona.csv', index=False)